In [1]:
import pandas as pd 


In [2]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding = 'latin1')

In [3]:
df.rename(columns = {'v1' : "label" , 'v2':'message'} , inplace = True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       5572 non-null   object
 1   message     5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [5]:
df.columns

Index(['label', 'message', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [6]:
df.drop(columns  = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] , axis = 1 , inplace = True)

In [7]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
import nltk 
from nltk.corpus  import stopwords
from nltk.stem.porter import PorterStemmer  
import re
stem = PorterStemmer()

In [9]:
corpus = []


for i in range(0 , len(df)): 
    review  = re.sub('[^a-zA-Z]' , " " , df['message'][i])
    review = review.lower()
    review = review.split()
    review = [stem.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vectors = TfidfVectorizer(max_features=5000 , ngram_range=(1, 2))

In [12]:
X = vectors.fit_transform(corpus).toarray()

In [13]:
y = pd.get_dummies(df['label'])

In [14]:
y = y.iloc[: , 0]

In [15]:
from sklearn.model_selection import train_test_split 

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.3 , random_state=42)

In [16]:
from sklearn.naive_bayes import MultinomialNB 

model = MultinomialNB()
model.fit(X_train , y_train)

MultinomialNB()

In [17]:
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score , classification_report


accuracy_score(y_test , y_pred)

0.9706937799043063

In [19]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

       False       0.99      0.78      0.87       219
        True       0.97      1.00      0.98      1453

    accuracy                           0.97      1672
   macro avg       0.98      0.89      0.93      1672
weighted avg       0.97      0.97      0.97      1672

